## Configuración y librerías
Asegúrate de ejecutar las celdas de carga/preprocesamiento equivalentes a `b_08_ml` antes de estas secciones (variables como `T`, `mu`, `sigma`, `ages`, `years`, `mat`, `mat_log`, etc.).

In [1]:
# Librerías básicas
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# Ajustes de visualización
sns.set(context='notebook', style='whitegrid')
np.set_printoptions(precision=6, suppress=True)

# Directorio de salida
OUT_DIR = os.environ.get('OUT_DIR', 'modelos/outputs/output')
os.makedirs(OUT_DIR, exist_ok=True)
print('OUT_DIR =', OUT_DIR)

/Users/scuartasr/Documents/Maestría/Tesis/tfm_tuberc/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OUT_DIR = modelos/outputs/output


## Funciones utilitarias (métricas)
Mismas métricas que en `b_08_ml`.

In [2]:
def mse(a, b):
    return float(np.mean((a - b) ** 2))

def rmse(a, b):
    return float(np.sqrt(mse(a, b)))

def mae(a, b):
    return float(np.mean(np.abs(a - b)))

def mape(a, b):
    eps = 1e-12
    return float(np.mean(np.abs((a - b) / (a + eps))) * 100)

def smape(a, b):
    denom = (np.abs(a) + np.abs(b))
    return float(np.mean(2.0 * np.abs(a - b) / np.where(denom == 0, 1.0, denom)) * 100)

def wape(a, b):
    return float(np.sum(np.abs(a - b)) / (np.sum(np.abs(a)) + 1e-12) * 100)

def rmse_log(a, b):
    return float(np.sqrt(np.mean((np.log1p(a) - np.log1p(b)) ** 2)))

## Construcción de entradas APC
Reutiliza la misma lógica que en `b_08_ml` para construir `X_train_apc`, `y_train`, etc. Aquí se define una función de ayuda que espera variables ya presentes en el entorno.

In [3]:
def build_apc_inputs(mat_log, ages, years, T, mu, sigma):
    # Ejemplo mínimo; ajusta según tu versión de b_08_ml.
    # Construye canales: logm_norm; feature_period_norm; feature_cohort_norm; age_mid_norm; target_period_norm
    N = (mat_log.shape[1] - T) * mat_log.shape[0]
    C = 5
    X = np.zeros((N, T, C), dtype=np.float32)
    y = np.zeros((N,), dtype=np.float32)
    meta = []
    idx = 0
    ages_mid = np.array([(a[0] + a[1]) / 2.0 for a in ages], dtype=float)
    ages_mid_norm = (ages_mid - ages_mid.mean()) / (ages_mid.std() + 1e-12)
    years_arr = np.array(years, dtype=float)
    years_norm = (years_arr - years_arr.mean()) / (years_arr.std() + 1e-12)
    for ai in range(mat_log.shape[0]):
        for t0 in range(mat_log.shape[1] - T):
            window = mat_log[ai, t0:t0+T]
            target = mat_log[ai, t0+T]
            # Canal 0: logm_norm
            X[idx, :, 0] = (window - mu) / (sigma + 1e-12)
            # Canal 1: feature_period_norm (años de las features)
            X[idx, :, 1] = years_norm[t0:t0+T]
            # Canal 2: feature_cohort_norm (cohortes aproximadas)
            cohorts = years_arr[t0:t0+T] - ages_mid[ai]
            cohorts_norm = (cohorts - cohorts.mean()) / (cohorts.std() + 1e-12)
            X[idx, :, 2] = cohorts_norm
            # Canal 3: age_mid_norm (constante por edad)
            X[idx, :, 3] = ages_mid_norm[ai]
            # Canal 4: target_period_norm (año del objetivo)
            X[idx, :, 4] = years_norm[t0+T-1]  # último año en ventana
            y[idx] = (target - mu) / (sigma + 1e-12)
            meta.append((ai, years_arr[t0+T]))
            idx += 1
    return X, y, meta

# Ejemplo de uso (descomenta cuando tengas las variables en el entorno):
# X_train_apc, y_train, meta_train = build_apc_inputs(mat_log_train, ages, years_train, T, mu, sigma)
# X_val_apc, y_val, meta_val = build_apc_inputs(mat_log_val, ages, years_val, T, mu, sigma)

## Modelo RNN (sustituto de CNN)
Se definen variantes con `SimpleRNN` y `LSTM`. Usa la que prefieras.

In [4]:
def build_rnn_apc(T, C, rnn_type='simple', rnn_units=64, dense_units=64, learning_rate=1e-3, dropout=0.0):
    inputs = layers.Input(shape=(T, C))
    if rnn_type == 'simple':
        x = layers.SimpleRNN(rnn_units, return_sequences=True)(inputs)
    elif rnn_type == 'lstm':
        x = layers.LSTM(rnn_units, return_sequences=True)(inputs)
    elif rnn_type == 'gru':
        x = layers.GRU(rnn_units, return_sequences=True)(inputs)
    else:
        raise ValueError('rnn_type debe ser simple|lstm|gru')
    if dropout > 0:
        x = layers.Dropout(dropout)(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(dense_units, activation='relu')(x)
    outputs = layers.Dense(1, activation='linear')(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    opt = optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss='mse')
    return model

# Ejemplo de instanciación (ajusta T, C):
# model_apc = build_rnn_apc(T=T, C=X_train_apc.shape[2], rnn_type='lstm', rnn_units=64, dense_units=64, learning_rate=1e-3)
# model_apc.summary()

## Entrenamiento (EarlyStopping)
Usa la misma configuración que en `b_08_ml`, cambiando únicamente el constructor del modelo.

In [5]:
# from tensorflow.keras.callbacks import EarlyStopping
# es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
# hist = model_apc.fit(X_train_apc, y_train, validation_data=(X_val_apc, y_val),
#                      epochs=200, batch_size=256, callbacks=[es], verbose=1)
# pd.DataFrame(hist.history).to_csv(os.path.join(OUT_DIR, 'hist_rnn_apc.csv'), index=False)

## SHAP por canal (APC)
Igual que en `b_08_ml`: se aplanan entradas (T*C), se envuelven las predicciones y se agregan por canal.

In [6]:
# Ejemplo de SHAP (ajusta a tu entorno de datos):
# def predict_flat(X_flat):
#     N = X_flat.shape[0]
#     X_seq = X_flat.reshape(N, T, X_train_apc.shape[2])
#     return model_apc.predict(X_seq, verbose=0).ravel()
# 
# X_bg = X_train_apc.reshape(X_train_apc.shape[0], -1)
# explainer = shap.Explainer(predict_flat, X_bg, algorithm='permutation')
# shap_vals = explainer(X_bg)
# shap_per_channel = np.mean(np.abs(shap_vals.values.reshape(X_train_apc.shape[0], T, X_train_apc.shape[2])), axis=(0,1))
# pd.DataFrame({'canal': ['logm','period','cohort','age_mid','target_period'], 'mean_abs_shap': shap_per_channel}).to_csv(os.path.join(OUT_DIR, 'shap_channel_importance_rnn_apc.csv'), index=False)
# plt.figure(figsize=(6,3)); plt.bar(['logm','period','cohort','age_mid','target_period'], shap_per_channel); plt.tight_layout(); plt.savefig(os.path.join(OUT_DIR, 'shap_channel_importance_rnn_apc.png'), dpi=160); plt.close()

## Validación y métricas
Replica la validación (TRAIN+VAL) y los heatmaps del período completo como en `b_08_ml`.

In [7]:
# Ejemplo de impresión de métricas (rellena con tus arrays):
# print(f"Train | MSE: {mse(y_train_true, y_train_pred):.4e} | RMSE: {rmse(y_train_true, y_train_pred):.4e} | MAE: {mae(y_train_true, y_train_pred):.4e} | MAPE: {mape(y_train_true, y_train_pred):.2f}% | sMAPE: {smape(y_train_true, y_train_pred):.2f}% | WAPE: {wape(y_train_true, y_train_pred):.2f}% | RMSE_log: {rmse_log(y_train_true, y_train_pred):.4f}")
# print(f"Val   | MSE: {mse(y_val_true, y_val_pred):.4e} | RMSE: {rmse(y_val_true, y_val_pred):.4e} | MAE: {mae(y_val_true, y_val_pred):.4e} | MAPE: {mape(y_val_true, y_val_pred):.2f}% | sMAPE: {smape(y_val_true, y_val_pred):.2f}% | WAPE: {wape(y_val_true, y_val_pred):.2f}% | RMSE_log: {rmse_log(y_val_true, y_val_pred):.4f}")

In [12]:
# Configuración de entorno y datos (idéntico a b_08_ml, pero independiente)
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
sns.set(context='notebook', style='whitegrid')
np.set_printoptions(precision=6, suppress=True)

# Detecta la raíz del proyecto (dos niveles arriba de modelos/notebooks)
_cwd = os.getcwd()
if _cwd.endswith(os.path.join('modelos', 'notebooks')):
    PROJ_ROOT = os.path.dirname(os.path.dirname(_cwd))
else:
    PROJ_ROOT = _cwd
DATA_PATH = os.path.join(PROJ_ROOT, 'data')
NOTEBOOK_NAME = 'b_09_ml'
OUT_DIR = os.path.join(PROJ_ROOT, 'modelos', 'outputs', 'output')
os.makedirs(OUT_DIR, exist_ok=True)
print('PROJ_ROOT =', PROJ_ROOT)
print('OUT_DIR   =', OUT_DIR)
print('DATA_PATH =', DATA_PATH)

# Carga de matrices y metadatos necesarios (replicar lógica de b_08_ml)
mat_path = os.path.join(DATA_PATH, 'processed', 'mortalidad', 'tasas_mortalidad_gret_per.csv')
if not os.path.exists(mat_path):
    raise FileNotFoundError(f'No se encontró el archivo esperado: {mat_path}')
df_long = pd.read_csv(mat_path)
print('Columnas CSV:', list(df_long.columns))

# Detecta nombres de columnas según variantes conocidas
col_age_candidates = ['grupo_etario', 'gr_et', 'grupo_et', 'edad_grupo', 'grupo']
col_year_candidates = ['periodo', 'anio', 'ano', 'year']
col_rate_candidates = ['tasa_mortalidad', 'tasa', 'rate']

def detect_col(cands):
    for c in cands:
        if c in df_long.columns:
            return c
    raise KeyError(f'No se encontró ninguna de las columnas esperadas: {cands}')

col_age = detect_col(col_age_candidates)
col_year = detect_col(col_year_candidates)
col_rate = detect_col(col_rate_candidates)
print('Usando columnas -> edad:', col_age, '| año:', col_year, '| tasa:', col_rate)

# Pivot a matriz edad x año de tasas por 100k
mat_obs_100k = df_long.pivot(index=col_age, columns=col_year, values=col_rate)
ages_sorted = mat_obs_100k.index.to_numpy()
years = mat_obs_100k.columns.to_numpy()
mat = mat_obs_100k.to_numpy(dtype=float)
mat_log = np.log(np.clip(mat, 1e-12, None))

# División TRAIN/VAL por corte de año (igual a b_08_ml)
cut_year = 2018
if cut_year not in years:
    # si el año 2018 no existe, usa el último año anterior a 2019
    years_sorted = np.sort(years)
    cut_year = years_sorted[years_sorted <= 2018][-1]
idx_cut = int(np.where(years == cut_year)[0][0])
mat_train = mat_log[:, :idx_cut+1]
mat_val = mat_log[:, idx_cut+1:]
train_years = years[:idx_cut+1]
val_years = years[idx_cut+1:]

# Normalización global del target (mu, sigma del TRAIN)
mu = float(np.mean(mat_train))
sigma = float(np.std(mat_train) + 1e-12)
print('mu, sigma =', mu, sigma)

# Parámetros de ventana T (igual que b_08_ml)
T = 8


PROJ_ROOT = /Users/scuartasr/Documents/Maestría/Tesis/tfm_tuberc
OUT_DIR   = /Users/scuartasr/Documents/Maestría/Tesis/tfm_tuberc/modelos/outputs/output
DATA_PATH = /Users/scuartasr/Documents/Maestría/Tesis/tfm_tuberc/data
Columnas CSV: ['ano', 't', 'gr_et', 'poblacion', 'conteo_defunciones', 'tasa_x100k', 'tasa']
Usando columnas -> edad: gr_et | año: ano | tasa: tasa
mu, sigma = -10.299621379372875 1.6257535810357813


In [13]:
# Construcción de entradas base y APC (idéntico en estructura a b_08_ml)
def build_base_inputs(mat_log, years, T, mu, sigma):
    N = (mat_log.shape[1] - T) * mat_log.shape[0]
    X = np.zeros((N, T, 1), dtype=np.float32)
    y = np.zeros((N,), dtype=np.float32)
    meta_age, meta_year = [], []
    idx = 0
    for ai in range(mat_log.shape[0]):
        for t0 in range(mat_log.shape[1] - T):
            window = mat_log[ai, t0:t0+T]
            target = mat_log[ai, t0+T]
            X[idx, :, 0] = (window - mu) / (sigma + 1e-12)
            y[idx] = (target - mu) / (sigma + 1e-12)
            meta_age.append(ai); meta_year.append(int(years[t0+T]))
            idx += 1
    return X, y, np.array(meta_age), np.array(meta_year)

def build_apc_inputs(mat_log, ages_idx_to_mid, years, T, mu, sigma):
    N = (mat_log.shape[1] - T) * mat_log.shape[0]
    C = 5
    X = np.zeros((N, T, C), dtype=np.float32)
    y = np.zeros((N,), dtype=np.float32)
    meta_age, meta_year = [], []
    years_arr = years.astype(float)
    years_norm = (years_arr - years_arr.mean()) / (years_arr.std() + 1e-12)
    ages_mid = np.array([ages_idx_to_mid[i] for i in range(mat_log.shape[0])], dtype=float)
    ages_mid_norm = (ages_mid - ages_mid.mean()) / (ages_mid.std() + 1e-12)
    idx = 0
    for ai in range(mat_log.shape[0]):
        for t0 in range(mat_log.shape[1] - T):
            window = mat_log[ai, t0:t0+T]
            target = mat_log[ai, t0+T]
            X[idx, :, 0] = (window - mu) / (sigma + 1e-12)
            X[idx, :, 1] = years_norm[t0:t0+T]
            cohorts = years_arr[t0:t0+T] - ages_mid[ai]
            cohorts_norm = (cohorts - cohorts.mean()) / (cohorts.std() + 1e-12)
            X[idx, :, 2] = cohorts_norm
            X[idx, :, 3] = ages_mid_norm[ai]
            X[idx, :, 4] = years_norm[t0+T-1]
            y[idx] = (target - mu) / (sigma + 1e-12)
            meta_age.append(ai); meta_year.append(int(years[t0+T]))
            idx += 1
    return X, y, np.array(meta_age), np.array(meta_year)

# Mapear índices de grupo etario a punto medio (asumiendo etiquetas tipo "[x,y)")
def parse_age_mid(age_label):
    # Ajusta si el formato difiere
    if isinstance(age_label, str) and ',' in age_label:
        parts = age_label.strip('[]()').split(',')
        a0, a1 = float(parts[0]), float(parts[1])
        return (a0 + a1) / 2.0
    try:
        return float(age_label)
    except:
        return np.nan

ages_idx_to_mid = {i: parse_age_mid(lbl) for i, lbl in enumerate(ages_sorted)}


In [14]:
# Construcción de datasets TRAIN/VAL
X_train_b, y_train_b, meta_age_b, meta_year_b = build_base_inputs(mat_train, train_years, T, mu, sigma)
X_val_b,   y_val_b,   meta_age_vb, meta_year_vb = build_base_inputs(mat_val,   val_years,   T, mu, sigma)

X_train_apc, y_train, meta_age_apc, meta_year_apc = build_apc_inputs(mat_train, ages_idx_to_mid, train_years, T, mu, sigma)
X_val_apc,   y_val,   meta_age_vapc, meta_year_vapc = build_apc_inputs(mat_val,   ages_idx_to_mid, val_years,   T, mu, sigma)
print('X_train_apc:', X_train_apc.shape, 'y_train:', y_train.shape)
print('X_val_apc:  ', X_val_apc.shape,   'y_val:  ', y_val.shape)
C = X_train_apc.shape[2]

ValueError: negative dimensions are not allowed

In [ ]:
# Definición del modelo RNN (sustituye CNN de b_08_ml)
def build_rnn_apc(T, C, rnn_type='lstm', rnn_units=64, dense_units=64, learning_rate=1e-3, dropout=0.0):
    inputs = layers.Input(shape=(T, C))
    if rnn_type == 'simple':
        x = layers.SimpleRNN(rnn_units, return_sequences=True)(inputs)
    elif rnn_type == 'lstm':
        x = layers.LSTM(rnn_units, return_sequences=True)(inputs)
    elif rnn_type == 'gru':
        x = layers.GRU(rnn_units, return_sequences=True)(inputs)
    else:
        raise ValueError('rnn_type debe ser simple|lstm|gru')
    if dropout > 0:
        x = layers.Dropout(dropout)(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(dense_units, activation='relu')(x)
    outputs = layers.Dense(1, activation='linear')(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    opt = optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss='mse')
    return model

from tensorflow.keras.callbacks import EarlyStopping
epochs = 200; batch_size = 256
es_apc = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
model_apc = build_rnn_apc(T=T, C=C, rnn_type='lstm', rnn_units=64, dense_units=64, learning_rate=1e-3, dropout=0.1)
model_apc.summary()
hist_apc = model_apc.fit(X_train_apc, y_train, validation_data=(X_val_apc, y_val), epochs=epochs, batch_size=batch_size, callbacks=[es_apc], verbose=1)
pd.DataFrame(hist_apc.history).to_csv(os.path.join(OUT_DIR, 'hist_rnn_apc.csv'), index=False)

In [ ]:
# Predicciones TRAIN y VAL, y métricas
yhat_train_scaled_apc = model_apc.predict(X_train_apc, verbose=0).ravel()
yhat_val_scaled_apc   = model_apc.predict(X_val_apc,   verbose=0).ravel()
ytrue_train_scaled_apc = y_train.ravel()
ytrue_val_scaled_apc   = y_val.ravel()

def print_metrics_split(y_true, y_pred, split):
    print(f"{split:<5} | MSE: {mse(y_true, y_pred):.4e} | RMSE: {rmse(y_true, y_pred):.4e} | MAE: {mae(y_true, y_pred):.4e} | MAPE: {mape(y_true, y_pred):.2f}% | sMAPE: {smape(y_true, y_pred):.2f}% | WAPE: {wape(y_true, y_pred):.2f}% | RMSE_log: {rmse_log(y_true, y_pred):.4f}")

print_metrics_split(ytrue_train_scaled_apc, yhat_train_scaled_apc, 'Train')
print_metrics_split(ytrue_val_scaled_apc,   yhat_val_scaled_apc,   'Val')

In [ ]:
# SHAP por canal para RNN-APC (idéntico patrón al de b_08_ml)
channel_names = ['logm_norm','feature_period_norm','feature_cohort_norm','age_mid_norm','target_period_norm']
X_train_flat = X_train_apc.reshape(X_train_apc.shape[0], -1)

def predict_flat(X_flat):
    N = X_flat.shape[0]
    X_seq = X_flat.reshape(N, T, C)
    return model_apc.predict(X_seq, verbose=0).ravel()

background_flat = X_train_flat
expl_apc = shap.Explainer(predict_flat, background_flat, algorithm='permutation')
sv_apc = expl_apc(X_train_flat)
shap_arr = sv_apc.values.reshape(X_train_apc.shape[0], T, C)
mean_abs_by_channel = np.mean(np.abs(shap_arr), axis=(0,1))
pd.DataFrame({'canal': channel_names, 'mean_abs_shap': mean_abs_by_channel}).to_csv(os.path.join(OUT_DIR, 'shap_channel_importance_rnn_apc.csv'), index=False)
plt.figure(figsize=(7,3)); plt.bar(channel_names, mean_abs_by_channel); plt.xticks(rotation=30); plt.tight_layout(); plt.savefig(os.path.join(OUT_DIR, 'shap_channel_importance_rnn_apc.png'), dpi=160); plt.close()

# Heatmap SHAP por tiempo y canal
mean_abs_by_t = np.mean(np.abs(shap_arr), axis=0)  # (T, C)
plt.figure(figsize=(8,4)); sns.heatmap(mean_abs_by_t.T, cmap='viridis', cbar=True, yticklabels=channel_names); plt.xlabel('Lag (t)'); plt.ylabel('Canal'); plt.tight_layout(); plt.savefig(os.path.join(OUT_DIR, 'shap_time_by_channel_rnn_apc.png'), dpi=160); plt.close()

In [ ]:
# Beeswarm APC (agregación por ejes Edad, Cohorte, Período)
# Construye una Explanation condensada con 3 features: Edad, Cohorte, Período
# Sumando contribuciones SHAP de canales correspondientes
def build_apc_axes_explanation(shap_arr, channel_names, X_train_flat):
    # Mapea canales a ejes:
    # Edad: age_mid_norm
    # Cohorte: feature_cohort_norm
    # Período: feature_period_norm + target_period_norm
    ch_idx = {name:i for i,name in enumerate(channel_names)}
    age_ch = ch_idx['age_mid_norm']
    cohort_ch = ch_idx['feature_cohort_norm']
    period_chs = [ch_idx['feature_period_norm'], ch_idx['target_period_norm']]
    # shap_arr shape: (N, T, C)
    age_shap = np.sum(np.abs(shap_arr[:,:,age_ch]), axis=1)    # (N,)
    cohort_shap = np.sum(np.abs(shap_arr[:,:,cohort_ch]), axis=1) # (N,)
    period_shap = np.sum(np.abs(shap_arr[:,:,period_chs]), axis=(1,2)) # (N,)
    values = np.stack([age_shap, cohort_shap, period_shap], axis=1) # (N,3)
    # Construir Explanation artificial
    feature_names = ['Grupo etario','Cohorte','Período']
    expl = shap.Explanation(values=values,
                             base_values=np.zeros(values.shape[0]),
                             data=X_train_flat[:,:3],
                             feature_names=feature_names)
    return expl

expl_apc_axes = build_apc_axes_explanation(shap_arr, channel_names, X_train_flat)
plt.figure(figsize=(8,4))
shap.plots.beeswarm(expl_apc_axes, max_display=3, color=None, show=False)
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, 'shap_beeswarm_rnn_apc_axes.png'), dpi=160)
plt.close()

In [ ]:
# Validación recursiva y heatmap completo 1979–2023 (100k)
# Reconstruye predicciones TRAIN (one-step) y VAL (recursiva) en escala original 100k
def inv_scale(y_scaled, mu, sigma):
    return y_scaled * sigma + mu
# Reconstruir matrices predichas a escala log y luego exponenciar a 100k
# TRAIN one-step
N_train = mat_train.shape[0] * (mat_train.shape[1] - T)
yhat_train_log = inv_scale(yhat_train_scaled_apc, mu, sigma)
# VAL recursiva: construir por edad, avanzando año a año
m_train_pred = np.full_like(mat_train, np.nan)
m_val_pred = np.full_like(mat_val, np.nan)
# Rellena TRAIN con últimos T para facilitar contextos
for ai in range(mat_train.shape[0]):
    for t0 in range(mat_train.shape[1] - T):
        tgt_idx = t0 + T
        m_train_pred[ai, tgt_idx] = yhat_train_log[ai*(mat_train.shape[1]-T) + t0]
# VAL recursiva
for ai in range(mat_val.shape[0]):
    # contexto inicial: últimos T puntos del TRAIN para esa edad
    context_log = mat_train[ai, -T:].copy()
    for t1 in range(mat_val.shape[1]):
        # features APC para la ventana actual
        win_years = val_years[max(0, t1-T+1):t1+1]
        # Construye secuencia (T,C) acorde a build_apc_inputs
        x_seq = np.zeros((1, T, C), dtype=np.float32)
        # Canal 0: logm_norm
        x_seq[0, :, 0] = (context_log - mu) / (sigma + 1e-12)
        # Canal 1: feature_period_norm
        years_arr = val_years.astype(float)
        years_norm = (years_arr - years_arr.mean()) / (years_arr.std() + 1e-12)
        t_start = max(0, t1-T+1)
        pad_len = T - (t1 - t_start + 1)
        fp = np.concatenate([np.zeros(pad_len), years_norm[t_start:t1+1]])
        x_seq[0, :, 1] = fp
        # Canal 2: feature_cohort_norm
        age_mid = ages_idx_to_mid[ai]
        cohorts = years_arr[t_start:t1+1] - age_mid
        if cohorts.size > 1:
            cohorts_norm = (cohorts - cohorts.mean()) / (cohorts.std() + 1e-12)
        else:
            cohorts_norm = np.zeros_like(cohorts)
        fc = np.concatenate([np.zeros(pad_len), cohorts_norm])
        x_seq[0, :, 2] = fc
        # Canal 3: age_mid_norm (constante)
        ages_mid_all = np.array([ages_idx_to_mid[i] for i in range(mat_train.shape[0])], dtype=float)
        age_mid_norm = (ages_mid_all - ages_mid_all.mean()) / (ages_mid_all.std() + 1e-12)
        x_seq[0, :, 3] = age_mid_norm[ai]
        # Canal 4: target_period_norm (año actual)
        x_seq[0, :, 4] = years_norm[max(0, t1-1)]
        yhat_scaled = model_apc.predict(x_seq, verbose=0).ravel()[0]
        yhat_log = inv_scale(yhat_scaled, mu, sigma)
        m_val_pred[ai, t1] = yhat_log
        # Actualiza contexto
        context_log = np.concatenate([context_log[1:], [yhat_log]])
# Convertir a tasas 100k
mat_train_pred_100k = np.exp(m_train_pred) * 1e5
mat_val_pred_100k   = np.exp(m_val_pred)   * 1e5
mat_obs_100k_train  = np.exp(mat_train)    * 1e5
mat_obs_100k_val    = np.exp(mat_val)      * 1e5
# Unir y graficar
mat_pred_full_100k = np.concatenate([mat_train_pred_100k, mat_val_pred_100k], axis=1)
mat_obs_full_100k  = np.concatenate([mat_obs_100k_train, mat_obs_100k_val], axis=1)
tick_years = years
fig, ax = plt.subplots(1, 2, figsize=(12, 5), constrained_layout=True)
vmin = np.nanmin(mat_obs_full_100k); vmax = np.nanmax(mat_obs_full_100k)
sns.heatmap(mat_obs_full_100k, ax=ax[0], cmap='magma', vmin=vmin, vmax=vmax, cbar=True)
ax[0].set_title('Observados (100k)'); ax[0].set_xlabel('Año'); ax[0].set_ylabel('Grupo etario')
sns.heatmap(mat_pred_full_100k, ax=ax[1], cmap='magma', vmin=vmin, vmax=vmax, cbar=True)
ax[1].set_title('Predichos RNN-APC (100k)'); ax[1].set_xlabel('Año'); ax[1].set_ylabel('Grupo etario')
plt.savefig(os.path.join(OUT_DIR, 'rnn_apc_matrices_obs_pred_1979_2023_100k.png'), dpi=160)
plt.close()